In [0]:
!pip install tensorflow==1.5.0

     |████████████████████████████████| 44.4MB 91kB/s 
     |████████████████████████████████| 3.0MB 46.2MB/s 
     |████████████████████████████████| 890kB 46.7MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=a21ea316e4e4e63f9c1434faf27d6f03983b6812f6a43dcde1e2485ab602e2e6
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('mnist_data', one_hot=True)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:496: FutureWarning: Passing (type, 1) or 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
num_steps = 500
batch_size = 128

dim_image = 784
nHL_G = 256
nHL_D = 256
dim_noise = 100

In [0]:
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

In [0]:
W = {
    'HL_G': tf.Variable(glorot_init([dim_noise, nHL_G])),
    'OL_G': tf.Variable(glorot_init([nHL_G, dim_image])),
    'HL_D': tf.Variable(glorot_init([dim_image, nHL_D])),
    'OL_D': tf.Variable(glorot_init([nHL_D, 1]))
}
b = {
    'HL_G': tf.Variable(tf.zeros([nHL_G])),
    'OL_G': tf.Variable(tf.zeros([dim_image])),
    'HL_D': tf.Variable(tf.zeros([nHL_D])),
    'OL_D': tf.Variable(tf.zeros([1]))
}

In [0]:
def nn_G(x):
    HL = tf.nn.relu(tf.add(tf.matmul(x, W['HL_G']), b['HL_G']))
    OL = tf.nn.sigmoid(tf.add(tf.matmul(HL, W['OL_G']), b['OL_G']))
    return OL

In [0]:
def nn_D(x):
    HL = tf.nn.relu(tf.add(tf.matmul(x, W['HL_D']), b['HL_D']))
    OL = tf.nn.sigmoid(tf.add(tf.matmul(HL, W['OL_D']), b['OL_D']))
    return OL

In [0]:
IN_G = tf.placeholder(tf.float32, shape=[None, dim_noise])
IN_D = tf.placeholder(tf.float32, shape=[None, dim_image])

In [0]:
sample_G = nn_G(IN_G)

D_real = nn_D(IN_D)
D_fake = nn_D(sample_G)
vars_G = [W['HL_G'], W['OL_G'], b['HL_G'], b['OL_G']]
vars_D = [W['HL_D'], W['OL_D'], b['HL_D'], b['OL_D']]

In [0]:
cost_G = -tf.reduce_mean(tf.log(D_fake))
cost_D = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
train_G = tf.train.AdamOptimizer(0.0002).minimize(cost_G, var_list=vars_G)
train_D = tf.train.AdamOptimizer(0.0002).minimize(cost_D, var_list=vars_D)

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1, num_steps+1):
        batch_images, _ = mnist.train.next_batch(batch_size)
        z = np.random.uniform(-1., 1., size=[batch_size, dim_noise])

        sess.run([train_G, train_D], feed_dict={IN_D: batch_images, IN_G: z})
        f, a = plt.subplots(4, 10, figsize=(10, 4))
        for i in range(10):
            z = np.random.uniform(-1., 1., size=[4, dim_noise])
            g = sess.run(sample_G, feed_dict={IN_G: z})
            g = np.reshape(g, newshape=(4, 28, 28, 1))
            g = -1 * (g - 1)
            for j in range(4):
                img = np.reshape(np.repeat(g[j][:, :, np.newaxis], 3, axis=2), newshape=(28, 28, 3))
                a[j][i].imshow(img)
    f.show()
    plt.draw()
    #plt.waitforbuttonpress()